In [16]:
import numpy as np
from PIL import Image  
import os, sys
import albumentations as A
from IPython.display import clear_output

In [17]:
!wget https://www.ispras.ru/conf/vessels/AngioCells.zip
!unzip AngioCells
!rm AngioCells.zip

clear_output()

In [18]:
def change_colors(*xx, old_colors, new_colors):
    assert len(old_colors) == len(new_colors)
    for x in xx:
        for old_color, new_color in zip(old_colors, new_colors):
            x[(x == np.array(old_color)).all(axis=2)] = np.array(new_color)
            
def change_pix(im, blue=0):
    arr = np.ones(shape=im.shape[:2], dtype='uint8')*2
    arr[(im==[0, 0, 0]).all(axis=2)] = 0 # backgrounds
    arr[(im==[59, 164, 118]).all(axis=2)] = 2 # tubes
    arr[(im==[174,  88, 119]).all(axis=2)] = 1 # nodes
    arr[(im==[50, 183, 250]).all(axis=2)] = blue # undefined
    return arr

In [19]:
tubes = (59,164,118) # green
nodes = (174,88,119) # red
backgrounds = (0, 0, 0) # black
undefined = (50,183,250) # blue

TARGET_SHAPE = (640, 480)

In [20]:
path_to_angiocells = './AngioCells' # insert your path to dataset

In [21]:
types = ['Dark', 'Defective', 'Different', 'Good']

for type in types:
    print(type)
    images_dir = f'{path_to_angiocells}/dataset/{type}/images/'
    masks_dir = f'{path_to_angiocells}/dataset/{type}/masks/'

    # Masks for agreement measurements
    def_masks_dir = f'{path_to_angiocells}/prepared_dataset/{type}/defined_masks/' 
    # res_images_dir contains resized images
    res_images_dir = f'{path_to_angiocells}/prepared_dataset/{type}/images/'
    # res_masks_dir contains resized masks
    res_masks_dir = f'{path_to_angiocells}/prepared_dataset/{type}/resized_masks/'
    # resized_mask contains masks with {0, 1, 2} pixels
    new_masks_dir = f'{path_to_angiocells}/prepared_dataset/{type}/masks/' 

    if not os.path.exists(def_masks_dir):
        os.makedirs(def_masks_dir)

    if not os.path.exists(new_masks_dir):
        os.makedirs(new_masks_dir)

    if not os.path.exists(res_images_dir):
        os.makedirs(res_images_dir)

    # set the undefined color to tubes' color
    for images in os.listdir(masks_dir):
        if (images.endswith('.png')):
            mask = np.array(Image.open(masks_dir + images))
            change_colors(mask, 
                        old_colors=(undefined, nodes, backgrounds), 
                        new_colors=(tubes, nodes, backgrounds),
                        )
            new_mask = Image.fromarray(np.array(mask, dtype='uint8'))
            new_mask.save(def_masks_dir+images)

    if not os.path.exists(res_masks_dir):
        os.makedirs(res_masks_dir)

    for images, masks in zip(sorted(os.listdir(images_dir)), sorted(os.listdir(def_masks_dir))):
        if (images.endswith('.png') or images.endswith('.jpg')) and (masks.endswith('.png') or masks.endswith('.jpg')):
            image = np.array(Image.open(images_dir + images), dtype = 'uint8')
            mask = np.array(Image.open(def_masks_dir + masks), dtype = 'uint8')
            resized_image_and_mask = A.Resize( TARGET_SHAPE[1], TARGET_SHAPE[0])(image=image, mask=mask)
            resized_image, resized_mask = resized_image_and_mask['image'], resized_image_and_mask['mask']
            
            fin_resized_image = Image.fromarray(resized_image) 
            fin_resized_image.save(res_images_dir+images)
            
            fin_resized_mask = Image.fromarray(resized_mask)
            fin_resized_mask.save(res_masks_dir+masks)
            
    # changing pixel color to 0, 1, 2 in masks
    for new_images in os.listdir(res_masks_dir):
        if (new_images.endswith('.png')):
            new_mask = np.array(Image.open(res_masks_dir + new_images))
            new_mask = change_pix(new_mask)  
            prepared_mask = Image.fromarray(np.array(new_mask, dtype='uint8'))
            prepared_mask.save(new_masks_dir+new_images)

Dark
Defective
Different
Good


In [22]:
'''
preparing masks for agreement measurement 
(coloring undefined blue areas as in the code above)

The operation does in-place
'''
experiments = ['phase_1', 'phase_2', 'phase_3', 'phase_3*']
for experiment in experiments:
    kappa_masks_dir = f'{path_to_angiocells}/agreement/{experiment}/'
    for images in os.listdir(kappa_masks_dir):
        if (images.endswith('.png')):
            mask = np.array(Image.open(kappa_masks_dir + images))
            change_colors(mask, 
                        old_colors=(undefined, nodes, backgrounds), 
                        new_colors=(tubes, nodes, backgrounds),
                        )
            new_mask = Image.fromarray(np.array(mask, dtype='uint8'))
            new_mask.save(kappa_masks_dir+images)